# Plot computed wave data

Python version of `sim_data_BM1/comp_wave_data.m`.

Work in progress for [NTHMP Debris Modeling Benchmarking Workshop](http://tsunamiworkshop.org/index.html), May 2023, by the [UW Tsunami Modeling Group](http://depts.washington.edu/ptha/).

Requires the `sim_data_BM1` directory available from http://tsunamiworkshop.org/problems/prob1.html

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
import os
import netCDF4
from scipy.interpolate import RegularGridInterpolator

In [ ]:
sim_data_dir = 'sim_data_BM1'

In [ ]:
eta_fname = os.path.join(sim_data_dir, 'zeta.nc')
f = netCDF4.Dataset(eta_fname, 'r')
time_numerical = array(f.variables['time'])
x_numerical = array(f.variables['x'])
zeta = squeeze(array(f.variables['zeta']))  # squeeze to get rid of y-dimension index
#depth = array(f.variables['depth'])

# numerical shift values - numerical domain was not same as physical domain
x_shift=-10.  # added 10-m to offshore boundary to create wave
t_shift=20-.74  # cut off first 20 ish seconds since nothing happened

time=time_numerical+t_shift
x=x_numerical+x_shift

In [ ]:
velo_fname = os.path.join(sim_data_dir, 'velo.nc')
f = netCDF4.Dataset(velo_fname, 'r')
u_vel = squeeze(array(f.variables['u_velo']))  # squeeze to get rid of y-dimension index

In [ ]:
print('time.shape = ',time.shape)
print('x.shape = ',x.shape)
print('zeta.shape = ',zeta.shape)
print('u_vel.shape = ',u_vel.shape)

#### Bilinear functions interpolating zeta and u_vel data:

In [ ]:
zeta_fcn = RegularGridInterpolator((time,x), zeta, method='linear')
u_vel_fcn = RegularGridInterpolator((time,x), u_vel, method='linear')

### offshore wave gages

In [ ]:
WG2_6 = loadtxt(os.path.join(sim_data_dir, 'WG2_6.txt')) 

WG2_loc = 2.26
tx = vstack((time, WG2_loc*ones(time.shape))).T
WG2_num = zeta_fcn(tx)

WG6_loc = 19.22
tx = vstack((time, WG6_loc*ones(time.shape))).T
WG6_num = zeta_fcn(tx)

figure(figsize=(10,6))
plot(WG2_6[:,0],WG2_6[:,1] , 'r-', label='WG2 Data')
plot(WG2_6[:,0],WG2_6[:,2], 'k-', label='WG6 Data')
plot(time, WG2_num, 'r--', label='WG2 Model')
plot(time, WG6_num, 'k--', label='WG6 Model')

legend()
xlabel('Time (s)')
ylabel('eta (m)')
axis([20, 55, -0.05, .5])
grid(True);
title('offshore wave gages');

### nearshore US wave gages

In [ ]:
USWG3 = loadtxt(os.path.join(sim_data_dir, 'USWG3.txt'))
USWG4 = loadtxt(os.path.join(sim_data_dir, 'USWG4.txt'))

USWG3_loc = 30.68
tx = vstack((time, USWG3_loc*ones(time.shape))).T
USWG3_num = zeta_fcn(tx)

USWG4_loc=31.89
tx = vstack((time, USWG4_loc*ones(time.shape))).T
USWG4_num = zeta_fcn(tx)

figure(figsize=(10,6))
plot(USWG3[:,0],USWG3[:,1] , 'r-', label='usWG3 Data')
plot(USWG4[:,0],USWG4[:,1], 'k-', label='usWG4 Data')
plot(time, USWG3_num-USWG3_num[1], 'r--', label='usWG3 Model')
plot(time, USWG4_num-USWG4_num[1], 'k--', label='usWG4 Model')

legend()
xlabel('Time (s)')
ylabel('eta (m)')
axis([20, 55, -0.05, .2])
grid(True);
title('nearshore US wave gages');

### nearshore ADV

In [ ]:
ADV2 = loadtxt(os.path.join(sim_data_dir, 'ADV2.txt'))

ADV2_loc = USWG4_loc
tx = vstack((time, ADV2_loc*ones(time.shape))).T
ADV2_num = u_vel_fcn(tx)

figure(figsize=(10,6))
plot(ADV2[:,0],ADV2[:,1] , 'b-', label='ADV2 Data')
plot(time, ADV2_num, 'k--', label='ADV2 Model')

legend()
xlabel('Time (s)')
ylabel('u (m/s)')
axis([20, 55, -1.2, 2.5])
grid(True);
title('nearshore ADV');

In [ ]:
zeta.shape